In [0]:

%pip install -U llama-index llama-index-llms-databricks mlflow
%restart_python

Note: you may need to restart the kernel using %restart_python or dbutils.library.restartPython() to use updated packages.


In [0]:
%pip install --upgrade databricks-vectorsearch databricks-genai-inference
%restart_python

In [0]:
	

# Configure your personal access token
from databricks.sdk import WorkspaceClient

w = WorkspaceClient()
tmp_token = w.tokens.create(comment="for model serving", lifetime_seconds=1200)

# Print what is the tmp token:
print(tmp_token.token_value)

[REDACTED]


In [0]:

from llama_index.llms.databricks import Databricks
from llama_index.core.agent.workflow import ReActAgent


llm = Databricks(
    model="databricks-llama-4-maverick",
    api_key=tmp_token.token_value,
    api_base=f"{w.config.host}/serving-endpoints/"
)

In [0]:
import pandas as pd

def format_context(df: pd.DataFrame) -> str:
    """
    Converts the DataFrame into a JSON string to ensure all data is passed
    to the model without truncation. JSON is also a great format for structured data
    like you have in 'description_by_sections'.
    """
    return df.to_json(orient='records', indent=2)

def find_accessible_airbnb_properties(location: str, ratings: float = 0, price: float = 10000000, guests: int = 1) -> str:
  """
  Queries the Bright Initiative Airbnb dataset for properties in a specific location. 
  Parameters:
  location: str, the full address
  ratings: float, the ratings of the property provided by customers
  price: float, the price of the property
  guests: the maximum number of the guest the property can accommodate
  """
  query = f"""
    SELECT
      listing_name,
      location_details,
      location,
      details,
      description_by_sections,
      reviews,
      amenities
    FROM dais_hackathon_shared_data.bright_initiative.airbnb_properties_information
    WHERE
      location ILIKE '%{location}%'
      and price <= {price}
      AND ratings >= {ratings}
      AND guests >= {guests}
      AND host_number_of_reviews > 1000
      AND EXISTS(reviews, review -> review ILIKE '%wheelchair%')
    LIMIT 5
  """
  return format_context(spark.sql(query).toPandas())

In [0]:
display(find_accessible_airbnb_properties("Chicago"))

'[\n  {\n    "listing_name":"Entire rental unit in Chicago, Illinois, United States",\n    "location_details":[\n      {\n        "title":"Neighbourhood highlights",\n        "value":"The South Loop is a vibrant blend of notable attractions, beautiful parks, and stunning Lake Michigan views. Picnic in the charming green space surrounding Museum Campus, or attend one of the many events at Grant Park such as The Taste of Chicago. Spend the day exploring the countless museums, or journey to the top of the 108-story Willis Tower for breathtaking views of the city."\n      }\n    ],\n    "location":"Chicago, Illinois, United States",\n    "details":[\n      "4 guests",\n      "2 bedrooms",\n      "2 beds",\n      "1 bath"\n    ],\n    "description_by_sections":[\n      {\n        "title":null,\n        "value":"With its ornate exterior and art-deco sign, The Plymouth is a South Loop staple. Through the glass doors and chic lobby, gold and wooden accents decorate the open space. You can loun

In [0]:
# View the response:
agent = ReActAgent(
    tools=[find_accessible_airbnb_properties],
    llm=llm,
    system_prompt="You are a helpful assistant that help people find the airbnb for accessible people.",
)

response = await agent.run("can you find an airbnb near Chicago with rating at least 3.8 and price below 400 per night.")

# View the response:
response.response

ChatMessage(role=<MessageRole.ASSISTANT: 'assistant'>, additional_kwargs={}, blocks=[TextBlock(block_type='text', text='I found two Airbnb properties in Chicago that meet your criteria. The first property is an entire rental unit in the South Loop area, accommodating 4 guests with 2 bedrooms and 2 beds. It has a rooftop with city skyline views and is fully equipped with amenities like a kitchen, in-suite laundry, and cable television. The second property is an entire condo, a studio apartment that is ADA accessible, located near the Blue Line L, making it convenient for getting to Downtown Chicago. It features a fully equipped kitchen, in-unit laundry, and spa baths. Both properties have high ratings and are priced below $400 per night.')])

In [0]:
response.response.blocks[0].text

'I found two Airbnb properties in Chicago that meet your criteria. The first property is an entire rental unit in the South Loop area, accommodating 4 guests with 2 bedrooms and 2 beds. It has a rooftop with city skyline views and is fully equipped with amenities like a kitchen, in-suite laundry, and cable television. The second property is an entire condo, a studio apartment that is ADA accessible, located near the Blue Line L, making it convenient for getting to Downtown Chicago. It features a fully equipped kitchen, in-unit laundry, and spa baths. Both properties have high ratings and are priced below $400 per night.'